## **1D FDTD Solver**

Python adaptation of John B. Schneider's C programs from chapter 3 of his textbook *Understanding the Finite-Difference Time-Domain
Method.*

Chapter 3 of Schneider's book is located [HERE](https://eecs.wsu.edu/~schneidj/ufdtd/chap3.pdf) and his GitHub source code is available [HERE](https://github.com/john-b-schneider/uFDTD).

### Animation Setup

Functions to create animations of the 1D FDTD solvers.

In [ ]:
%matplotlib inline
import matplotlib.animation as animation
import matplotlib.pyplot as plt
from IPython import display
from IPython.display import HTML

fig, ax1 = plt.subplots(dpi=144)
ax2 = ax1.twinx()
e_field, = ax1.plot([], [], "-b")
h_field, = ax2.plot([], [], "--r")
artist_list = []
fig.set_tight_layout(True)  # Needed to prevent right label being cut off


def anim_init():
    """Initialize plot for the animation."""
    for artist in artist_list:
        artist.remove()  # Clear any previously drawn rectangles
    artist_list.clear()  # Remove artists from list
    e_field.set_data([], [])
    h_field.set_data([], [])

    ax1.set_xlim(0, SIZE)
    ax1.set_ylim(-1, 1)
    ax2.set_ylim(-1 / 377, 1 / 377)
    ax1.set_xlabel("Node Number")
    ax1.set_ylabel("E-Field", color="b")
    ax2.set_ylabel("H-Field", color="r")
    ax2.yaxis.set_label_position("right")
    ax2.yaxis.tick_right()

    if sim_desc == 'dielectric' or sim_desc == 'lossy_dielectric':
        artist_list.append(ax1.axvspan(100, SIZE, facecolor="green",
                                        alpha=0.3))
    elif sim_desc == 'matched':
        artist_list.append(ax1.axvspan(100, LOSS_LAYER, facecolor="green",
                                        alpha=0.3))
        artist_list.append(ax1.axvspan(LOSS_LAYER, SIZE, facecolor="blue",
                                        alpha=0.3))

    return (e_field, h_field)


def animate(*args):
    """Draw the E-field and H-field magnitudes along X-axis at current time step."""
    ez, hy = next(sim_step)
    e_field.set_data(x, ez)
    h_field.set_data(x[: len(hy)], hy)
    return (e_field, h_field)


def html_video():
    """Jupyter notebook must have animation converted to HTML5 video to display."""
    mpl_animation = animation.FuncAnimation(
        fig, animate, frames=maxTime, init_func=anim_init, interval=25,
        blit=True
    )
    return mpl_animation


### Program 3.1

**1DbareBones.c:** Bare-bones one-dimensional simulation with a hard source.

In [ ]:
import numpy as np


def bare_bones(maxTime=250, SIZE=200):
    """Python adaptation of 1D FDTD "bare bones" simulation.

    ez[0] is a "hard source" that produces a Gaussian E-field pulse.  After sufficient time
    has passed it is forced to near-zero and becomes a perfect electric conductor (PEC).

    hy[SIZE-1] is forced to zero; a perfect magnetic conductor (PMC).
    """
    ez = np.zeros(SIZE)
    hy = np.zeros(SIZE)
    imp0 = 377  # Characteristic impedance of free space

    for qTime in range(maxTime):
        for mm in range(SIZE - 1):  # Update magnetic field
            hy[mm] = hy[mm] + (ez[mm + 1] - ez[mm]) / imp0
        for mm in range(1, SIZE):  # Update electric field
            ez[mm] = ez[mm] + (hy[mm] - hy[mm - 1]) * imp0
        ez[0] = np.exp(
            -(qTime - 30) ** 2 / 100
        )  # Gaussian E-field pulse source at first node
        # print(ez[50])
        yield ez, hy


In [ ]:
"""
Note that the H-field is reflected and inverted by the PMC at the final node.  
The E-field is reflected and inverted by the PEC at node 0.
"""

maxTime = 500  # Maximum number of simulated time steps
SIZE = 200  # Number of nodes along X-axis
x = np.arange(SIZE)

sim_step = bare_bones(maxTime, SIZE)
sim_desc = 'barebones'

anim = html_video()
HTML(anim.to_jshtml())


### Program 3.4

**1Dadditive.c:** One-dimensional FDTD program with an additive source.

In [ ]:
def additive(maxTime=250, SIZE=200):
    """Python adaptation of 1D FDTD additive source simulation.

    ez[50] is an "additive source" that produces a Gaussian E-field pulse.  Its 
    E-field contribution is simply added to the node after the E-field update step.

    hy[SIZE-1] is forced to zero; a perfect magnetic conductor (PMC).
    """
    ez = np.zeros(SIZE)
    hy = np.zeros(SIZE)
    imp0 = 377  # Characteristic impedance of free space

    for qTime in range(maxTime):
        for mm in range(SIZE - 1):  # Update magnetic field
            hy[mm] = hy[mm] + (ez[mm + 1] - ez[mm]) / imp0
        for mm in range(1, SIZE):  # Update electric field
            ez[mm] = ez[mm] + (hy[mm] - hy[mm - 1]) * imp0
        ez[50] += np.exp(
            -(qTime - 30) ** 2 / 100
        )  # Additive Gaussian E-field pulse source at node 50
        # print(ez[50])
        yield ez, hy


In [ ]:
maxTime = 250  # Maximum number of simulated time steps
SIZE = 200  # Number of nodes along X-axis
x = np.arange(SIZE)

sim_step = additive(maxTime, SIZE)
sim_desc = 'additive'

anim = html_video()
HTML(anim.to_jshtml())


### Program 3.4(b)

**1Dadditive.c with ABC:** One-dimensional FDTD program with an additive source and absorbing boundary condition (ABC).

In [ ]:
def additive_abc(maxTime=250, SIZE=200):
    """Python adaptation of 1D FDTD additive source simulation.

    ez[50] is an "additive source" that produces a Gaussian E-field pulse.  Its 
    E-field contribution is simply added to the node after the E-field update step.

    hy[SIZE-1] and ez[0] are absorbing boundary conditions (ABC).
    """
    ez = np.zeros(SIZE)
    hy = np.zeros(SIZE)
    imp0 = 377  # Characteristic impedance of free space

    for qTime in range(maxTime):
        hy[SIZE - 1] = hy[SIZE - 2]  # ABC
        for mm in range(SIZE - 1):  # Update magnetic field
            hy[mm] = hy[mm] + (ez[mm + 1] - ez[mm]) / imp0

        ez[0] = ez[1]  # ABC
        for mm in range(1, SIZE):  # Update electric field
            ez[mm] = ez[mm] + (hy[mm] - hy[mm - 1]) * imp0
        ez[50] += np.exp(
            -(qTime - 30) ** 2 / 100
        )  # Additive Gaussian E-field pulse source at node 50
        # print(ez[50])
        yield ez, hy


In [ ]:
maxTime = 250  # Maximum number of simulated time steps
SIZE = 200  # Number of nodes along X-axis
x = np.arange(SIZE)

sim_step = additive_abc(maxTime, SIZE)
sim_desc = 'additive_abc'

anim = html_video()
HTML(anim.to_jshtml())


### Program 3.5

**1Dtfsf.c:** One-dimensional simulation with a TFSF boundary between hy[49]
and ez[50].

In [ ]:
def tfsf_1d(maxTime=450, SIZE=200):
    """Python adaptation of 1D Total Field/Scattered Field boundary simulation.

    The TFSF boundary exists between hy[49] and ez[50].  The incident field
    only propagates to the right.

    hy[SIZE-1] and ez[0] are absorbing boundary conditions (ABC).
    """
    ez = np.zeros(SIZE)
    hy = np.zeros(SIZE)
    imp0 = 377  # Characteristic impedance of free space

    for qTime in range(maxTime):
        hy[SIZE - 1] = hy[SIZE - 2]  # ABC
        for mm in range(SIZE - 1):  # Update magnetic field
            hy[mm] = hy[mm] + (ez[mm + 1] - ez[mm]) / imp0
        hy[49] -= (
            np.exp(-(qTime - 30) ** 2 / 100) / imp0
        )  # Correction for Hy adjacent to TFSF boundary

        ez[0] = ez[1]  # ABC
        for mm in range(1, SIZE):  # Update electric field
            ez[mm] = ez[mm] + (hy[mm] - hy[mm - 1]) * imp0
        ez[50] += np.exp(
            -(qTime + 0.5 - (-0.5) - 30) ** 2 / 100
        )  # Correction for Ez adjacent to TFSF boundary

        yield ez, hy


In [ ]:
maxTime = 450  # Maximum number of simulated time steps
SIZE = 200  # Number of nodes along X-axis
x = np.arange(SIZE)

sim_step = tfsf_1d(maxTime, SIZE)
sim_desc = 'tfsf'

anim = html_video()
HTML(anim.to_jshtml())


### Program 3.6

**1Ddielectric.c:** One-dimensional FDTD program to model an interface between free-space and a dielectric that has a relative permittivity &epsilon;<sub>r</sub> of 9.

In [ ]:
def dielectric_1d(maxTime=450, SIZE=200):
    """Python adaptation of a TFSF simulation with a dielectric material starting at ez[100].

    A region with a relative dielectric constant of 9 begins at ez[100].

    The TFSF boundary exists between hy[49] and ez[50].  The incident field
    only propagates to the right.

    ez[SIZE-1] and ez[0] are absorbing boundary conditions (ABC).  
    The simulation begins and ends with an electric field node, and so Hy is of length SIZE-1.
    """
    ez = np.zeros(SIZE)
    hy = np.zeros(SIZE - 1)
    epsR = np.ones(SIZE)
    epsR[100:] = 9  # Create dielectric region
    imp0 = 377  # Characteristic impedance of free space

    for qTime in range(maxTime):
        for mm in range(SIZE - 1):  # Update magnetic field
            hy[mm] = hy[mm] + (ez[mm + 1] - ez[mm]) / imp0
        hy[49] -= (
            np.exp(-(qTime - 30) ** 2 / 100) / imp0
        )  # Correction for Hy adjacent to TFSF boundary

        ez[0] = ez[1]  # ABC
        ez[SIZE - 1] = ez[SIZE - 2]  # ABC
        for mm in range(1, SIZE - 1):  # Update electric field
            ez[mm] = ez[mm] + (hy[mm] - hy[mm - 1]) * imp0 / epsR[mm]
        ez[50] += np.exp(
            -(qTime + 0.5 - (-0.5) - 30) ** 2 / 100
        )  # Correction for Ez adjacent to TFSF boundary

        yield ez, hy


In [ ]:
maxTime = 500  # Maximum number of simulated time steps
SIZE = 200  # Number of nodes along X-axis
x = np.arange(SIZE)

sim_step = dielectric_1d(maxTime, SIZE)
sim_desc = 'dielectric'

anim = html_video()
HTML(anim.to_jshtml())


### Program 3.7
**1Dlossy.c:** One-dimensional simulation with a lossy dielectric region.

In [ ]:
def lossy_1d(maxTime=450, SIZE=200, LOSS=0.01):
    """Python adaptation of a TFSF simulation with a lossy dielectric material starting at ez[100].

    A region with a relative dielectric constant of 9 and loss of 0.01 begins at ez[100].

    The TFSF boundary exists between hy[49] and ez[50].  The incident field
    only propagates to the right.

    ez[0] is an absorbing boundary conditions (ABC).  
    The ABC at SIZE-1 has been removed and is a PMC. The loss is sufficient that the reflection will be negligible.
    The simulation begins and ends with an electric field node, and so Hy is of length SIZE-1.
    """
    ez = np.zeros(SIZE)
    hy = np.zeros(SIZE - 1)
    ceze = np.ones(SIZE)
    ceze[100:] = (1 - LOSS) / (
        1 + LOSS
    )  # Create lossy dielectric region; coefficient of E-fields
    imp0 = 377  # Characteristic impedance of free space
    cezh = imp0 * np.ones(SIZE)
    cezh[100:] = (
        imp0 / 9 / (1 + LOSS)
    )  # Create lossy dielectric region; coefficient of H-fields

    for qTime in range(maxTime):
        for mm in range(SIZE - 1):  # Update magnetic field
            hy[mm] = hy[mm] + (ez[mm + 1] - ez[mm]) / imp0
        hy[49] -= (
            np.exp(-(qTime - 30) ** 2 / 100) / imp0
        )  # Correction for Hy adjacent to TFSF boundary

        ez[0] = ez[1]  # ABC
        for mm in range(1, SIZE - 1):  # Update electric field
            ez[mm] = ceze[mm] * ez[mm] + cezh[mm] * (hy[mm] - hy[mm - 1])
        ez[50] += np.exp(
            -(qTime + 0.5 - (-0.5) - 30) ** 2 / 100
        )  # Correction for Ez adjacent to TFSF boundary

        yield ez, hy


In [ ]:
maxTime = 450  # Maximum number of simulated time steps
SIZE = 200  # Number of nodes along X-axis
x = np.arange(SIZE)

sim_step = lossy_1d(maxTime, SIZE)
sim_desc = 'lossy_dielectric'

anim = html_video()
HTML(anim.to_jshtml())


### Program 3.8

**1Dmatched.c:** Program with a lossless dielectric region followed by a lossy layer that has its impedance matched to the lossless dielectric.

In [ ]:
def matched_1d(maxTime=450, SIZE=200, LOSS=0.02, LOSS_LAYER=180):
    """Python adaptation of a TFSF simulation with a lossless dielectric material starting at ez[100] followed by a
    matched lossy dielectric region starting at ez[180].

    A region with a relative dielectric constant of 9 and loss of 0.01 begins at ez[100].

    The TFSF boundary exists between hy[49] and ez[50].  The incident field
    only propagates to the right.

    ez[0] is an absorbing boundary conditions (ABC).  
    The simulation begins and ends with an electric field node, and so Hy is of length SIZE-1.
    """
    ez = np.zeros(SIZE)
    hy = np.zeros(SIZE - 1)

    # Set electric field update coefficients
    ceze = np.ones(SIZE)
    ceze[LOSS_LAYER:] = (1 - LOSS) / (
        1 + LOSS
    )  # Create lossy dielectric region; coefficient of E-fields
    imp0 = 377  # Characteristic impedance of free space
    cezh = imp0 * np.ones(SIZE)
    cezh[100:LOSS_LAYER] = imp0 / 9  # Create lossless dielectric region
    cezh[LOSS_LAYER:] = (
        imp0 / 9 / (1 + LOSS)
    )  # Create lossy dielectric region; coefficient of H-fields

    # Set magnetic field update coefficients
    chyh = np.ones(SIZE - 1)
    chyh[LOSS_LAYER:] = (1 - LOSS) / (
        1 + LOSS
    )  # Lossy dielectric region; H-field coefficient
    chye = np.ones(SIZE - 1) / imp0
    # Lossy dielectric region, E-field coefficient
    chye[LOSS_LAYER:] /= 1 + LOSS

    # Time stepping
    for qTime in range(maxTime):
        for mm in range(SIZE - 1):  # Update magnetic field
            hy[mm] = chyh[mm] * hy[mm] + chye[mm] * (ez[mm + 1] - ez[mm])
        hy[49] -= (
            np.exp(-(qTime - 30) ** 2 / 100) / imp0
        )  # Correction for Hy adjacent to TFSF boundary

        ez[0] = ez[1]  # ABC
        for mm in range(1, SIZE - 1):  # Update electric field
            ez[mm] = ceze[mm] * ez[mm] + cezh[mm] * (hy[mm] - hy[mm - 1])
        ez[50] += np.exp(
            -(qTime + 0.5 - (-0.5) - 30) ** 2 / 100
        )  # Correction for Ez adjacent to TFSF boundary

        yield ez, hy


In [ ]:
maxTime = 450  # Maximum number of simulated time steps
SIZE = 200  # Number of nodes along X-axis
LOSS_LAYER = 180  # Start of matched lossy layer
x = np.arange(SIZE)

sim_step = matched_1d(maxTime, SIZE)
sim_desc = 'matched'

anim = html_video()
HTML(anim.to_jshtml())
